In [231]:
import itertools as it
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

Possible strategies:
        -- Need to be observed to counter:

- fixed sequence of 'n' moves
        -- previous 'n'+1 moves

- (opponent's / self's) last 'n'[0-10] moves
        -- previous n moves

- (opponent's / self's) most common move in the last 'n'[2-1000] matches
        -- ...

- (opponent's / self's) most common 'm-moves'[2-10] sequence in the last 'n'[2-1000] matches
        -- ...

In [232]:
#variables:
length = 10 #number of previous matches to analyze
seq_size = range(2, 6) #sequence sizes to test
matches_range = [5, 10] #number of previous matches to look for moves

In [233]:
## Some functions that will be used in the main script ##

#function for counting the number of occurences of a substring in a string
def count_substring(s, sub):
    # Initialize count to zero
    cnt = 0

    # Iterate through the string to check for the substring
    for i in range(len(s) - len(sub) + 1):
        if s[i:i + len(sub)] == sub:  # If substring matches, increment count
            cnt += 1
    return cnt

In [234]:
opponent_hist = "RRRPPPSSS" * 10
player_hist = "RPS" * 30
player_hist

'RPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPS'

In [235]:
vocab_moves = sorted(['R', 'P', 'S'])
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab_moves)}
idx2char = np.array(vocab_moves)

def text_to_int(text):
  return [char2idx[c] for c in text]

def int_to_text(ints):
  return ''.join(idx2char[ints])

In [236]:
#vocab of sequences with 2-5 elements
vocab_seq = {}
for i in seq_size:
  vocab_seq[i] = []
  for seq in it.product(vocab_moves, repeat=i):
    vocab_seq[i].append(''.join(seq))

#full vocabulary
vocab = vocab_moves.copy()
for i in seq_size:
  for seq in it.product(vocab_moves, repeat=i):
    vocab.append(''.join(seq))

def int_encoding(text):
  return vocab.index(text)

def int_decoding(ints):
  return vocab[ints]

In [237]:
#crete a list with the last 'length' moves
#of the opponent
op_last_moves = {}
for i in range(length):
    op_last_moves[i] = []
    for j in range(len(opponent_hist) - length):
        op_last_moves[i].append(opponent_hist[i + j])

#of the player
pl_last_moves = {}
for i in range(length):
    pl_last_moves[i] = []
    for j in range(len(player_hist) - length):
        pl_last_moves[i].append(player_hist[i + j])

#opponent_last_moves = [opponent_hist[i:i+length] for i in range(len(opponent_hist) - length)]
#player_last_moves = [player_hist[i:i+length] for i in range(len(opponent_hist) - length)]
#last_moves = opponent_last_moves + player_last_moves

#op_last_moves[0]

In [238]:
#create a list with the most common move in the last 'matches_range' matches, and in all matches
#of the opponent
op_commons = {}
for i in matches_range:
  op_commons[i] = []
  for j in range(length, len(opponent_hist)):
    op_commons[i].append(max(set(opponent_hist[j-i:j]), key=opponent_hist[j-i:j].count))
op_commons['all'] = []
for i in range(length, len(opponent_hist)):
  op_commons['all'].append(max(set(opponent_hist[:i]), key=opponent_hist[:i].count))

#of the player
pl_commons = {}
for i in matches_range:
  pl_commons[i] = []
  for j in range(length, len(opponent_hist)):
    pl_commons[i].append(max(set(player_hist[j-i:j]), key=player_hist[j-i:j].count))
pl_commons['all'] = []
for i in range(length, len(opponent_hist)):
  pl_commons['all'].append(max(set(player_hist[:i]), key=player_hist[:i].count))

#op_commons['all']

In [239]:
#create a list with the last sequence of 'seq_size' moves
#of the opponent
op_last_seqs = {}
for i in seq_size:
  op_last_seqs[i] = [opponent_hist[j-i:j] for j in range(length, len(opponent_hist))]

#of the player
pl_last_seqs = {}
for i in seq_size:
  pl_last_seqs[i] = [player_hist[j-i:j] for j in range(length, len(opponent_hist))]

#op_last_seqs[2]

In [240]:
#create a list with the most common sequence of 'seq_size' moves in the last 'matches_range' matches
#of the opponent
op_common_seqs = {}

for i in seq_size:
    op_common_seqs[i] = {}
    for j in matches_range:
        op_common_seqs[i][j] = []
        for k in range(length, len(opponent_hist)):
            count_subs = {}
            for seq in vocab_seq[i]:
                count_subs[seq] = count_substring(opponent_hist[k-j:k], seq)
            op_common_seqs[i][j].append(max(count_subs, key=count_subs.get))

#of the player
pl_common_seqs = {}

for i in seq_size:
    pl_common_seqs[i] = {}
    for j in matches_range:
        pl_common_seqs[i][j] = []
        for k in range(length, len(opponent_hist)):
            count_subs = {}
            for seq in vocab_seq[i]:
                count_subs[seq] = count_substring(player_hist[k-j:k], seq)
            pl_common_seqs[i][j].append(max(count_subs, key=count_subs.get))

#op_common_seqs[2][5]

In [241]:
#create a list with the most common sequence of 'seq_size' moves in all matches
#of the opponent
op_common_seqs_all = {}

for i in seq_size:
    op_common_seqs_all[i] = []
    for k in range(length, len(opponent_hist)):
        count_subs = {}
        for seq in vocab_seq[i]:
            count_subs[seq] = count_substring(opponent_hist[:k], seq)
        op_common_seqs_all[i].append(max(count_subs, key=count_subs.get))

#of the player
pl_common_seqs_all = {}

for i in seq_size:
    pl_common_seqs_all[i] = []
    for k in range(length, len(opponent_hist)):
        count_subs = {}
        for seq in vocab_seq[i]:
            count_subs[seq] = count_substring(player_hist[:k], seq)
        pl_common_seqs_all[i].append(max(count_subs, key=count_subs.get))

#op_common_seqs_all[2]

In [242]:
#join all previously defined features in a single list
features = []
for i in range(length):
    features.append(op_last_moves[i])
    features.append(pl_last_moves[i])
for i in op_commons.keys():
    features.append(op_commons[i])
    features.append(pl_commons[i])
for i in seq_size:
    features.append(op_last_seqs[i])
    features.append(pl_last_seqs[i])
for i in op_common_seqs.keys():
    for j in op_common_seqs[i].keys():
        features.append(op_common_seqs[i][j])
        features.append(pl_common_seqs[i][j])
for i in op_common_seqs_all.keys():
    features.append(op_common_seqs_all[i])
    features.append(pl_common_seqs_all[i])

print(features)

[['R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S'], ['R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P', 'S', 'R', 'P'], ['R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P', 'P', 'P', 'S', 'S', 'S', 'R', 'R', 'R', 'P',

In [216]:
#list of labels (correct next opponent's move on each match)
labels = []
for i in range(length, len(opponent_hist)):
    labels.append(opponent_hist[i])

#labels

In [217]:
#encode the features and labels. Transpose because the LSTM model expects the input in the shape (batch, timesteps, feature)
encoded_features = []
for i in range(len(features)):
    encoded_features.append([])
    for j in range(len(features[i])):
        encoded_features[i].append(int_encoding(features[i][j]))

encoded_features_np = np.array(encoded_features).transpose()
encoded_labels_np = np.array([int_encoding(label) for label in labels])

encoded_features_np.shape, encoded_features_np, encoded_labels_np.shape, encoded_labels_np, encoded_features_np[0], encoded_labels_np[0]

((80, 58),
 array([[  1,   1,   1, ...,  73, 128, 185],
        [  1,   0,   1, ...,  60, 128, 222],
        [  1,   2,   0, ...,  60, 128, 185],
        ...,
        [  0,   2,   2, ...,  60, 228, 185],
        [  2,   1,   2, ...,  73, 203, 185],
        [  2,   0,   2, ...,  60, 128, 222]]),
 (80,),
 array([1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0,
        0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2,
        1, 1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0,
        0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2]),
 array([  1,   1,   1,   0,   1,   2,   0,   1,   0,   0,   0,   2,   2,
          1,   2,   0,   2,   2,   1,   1,   2,   1,   1,   1,   1,   1,
         10,  10,  37,  19, 118,  73, 199, 316,  11,  10,   3,   5,  20,
         19,  12,  19,  65,  73,  41,  73, 199, 316, 128, 185,   3,   5,
         12,  19,  41,  73, 128, 185]),
 np.int64(1))

In [218]:
#one-hot encode the features and labels
onehot_features = tf.one_hot(encoded_features_np, len(vocab))
onehot_labels = tf.one_hot(encoded_labels_np, len(vocab_moves))
onehot_features.shape, onehot_labels.shape

(TensorShape([80, 58, 363]), TensorShape([80, 3]))

In [220]:
model = tf.keras.Sequential([
    layers.LSTM(1024, input_shape=(onehot_features.shape[1], onehot_features.shape[2])),
    layers.Dense(1024, activation='relu'),
    layers.Dense(onehot_labels.shape[1], activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['Accuracy'])

history = model.fit(
    onehot_features,
    onehot_labels,
    epochs=100)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 725ms/step - Accuracy: 0.5469 - loss: 1.0755
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 720ms/step - Accuracy: 0.4594 - loss: 0.9644
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 730ms/step - Accuracy: 0.8031 - loss: 0.9104
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 728ms/step - Accuracy: 0.8727 - loss: 0.7603
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 718ms/step - Accuracy: 0.6031 - loss: 1.0221
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 711ms/step - Accuracy: 0.8016 - loss: 0.7156
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 712ms/step - Accuracy: 0.8570 - loss: 0.6003
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 812ms/step - Accuracy: 0.9031 - loss: 0.4204
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 742ms/step - Accuracy: 0.9625 - loss: 0.3490
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 721ms/step - Accuracy: 0.7766 - loss: 0.3483
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 734ms/step - Accuracy: 0.7828 - loss: 0.3751
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 802ms/step - Accuracy:

In [198]:
def predict(opponent, player):
    # input: opponent and player histories
    # output: predicted output and it's probability
    m = min(len(opponent), len(player)) #if the inputs have different lengths, 'm' is the smaller length,
    seq = opponent[-m :] + player[-m :] #only use the last 'm' plays of each player
    seq_encoded = np.array([text_to_int(seq)]) #encode sequence of chars into sequence of ints
    seq_onehot = np.array([tf.keras.utils.to_categorical(se, num_classes=len(vocab)) for se in seq_encoded])
        #encode sequence of ints into sequence of one-hots
    pred = model.predict(seq_onehot) #run the prediction. Return list of probabilities for next char
    idx = np.argmax(pred) #idx of the char with max probability, equal to the integer encoding for the chars
    return int_to_text(idx), round(float(pred[0][idx]), 2) #predicted char, probability

predict('RPS'*5, 'RPS'*5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


('P', 0.93)